### Week 5 Day 4

AutoGen Core - Distributed

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..

In [16]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = True

### Start with our Message class

In [17]:

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

In [18]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

### Let's reintroduce a tool

In [19]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [20]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### And make some Agents

In [21]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        #model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        model_client = OpenAIChatCompletionClient(model="gemini-2.5-pro", model_info={
            "vision": True,
            "function_calling": True,
            "json_output": True,
            "structured_output": True,
            "family":"gemini-2.5-pro"
        })
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        #model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        model_client = OpenAIChatCompletionClient(model="gemini-2.5-pro", model_info={
            "vision": True,
            "function_calling": True,
            "json_output": True,
            "structured_output": True,
            "family":"gemini-2.5-pro"
        })
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        #model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        model_client = OpenAIChatCompletionClient(model="gemini-2.5-pro", model_info={
            "vision": True,
            "function_calling": True,
            "json_output": True,
            "structured_output": True,
            "family":"gemini-2.5-pro"
        })
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [22]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




In [23]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [24]:
display(Markdown(response.content))

## Pros of AutoGen:
Based on my research, here are the primary advantages and reasons to choose AutoGen for an AI Agent project:

*   **Advanced Multi-Agent Collaboration:** AutoGen excels at creating and orchestrating multiple, specialized AI agents that can work together to solve complex tasks. Instead of relying on a single, monolithic LLM, you can build a team of agents (e.g., a "planner," a "coder," a "critic") that converse and delegate to achieve a goal more effectively.

*   **Flexibility and Customization:** The framework is highly adaptable. You can customize agents for specific roles and capabilities, integrate various LLMs (not just OpenAI's), and equip them with different tools (like code interpreters or external APIs).

*   **Human-in-the-Loop Integration:** AutoGen makes it easy to integrate human oversight and feedback directly into the workflow. Agents can pause their work to ask for clarification, approval, or direction, allowing for a blend of autonomous operation and human control.

*   **Simplifies Complex Workflows:** It provides a high-level abstraction layer that simplifies the development of sophisticated agent-based applications. It handles the "plumbing" of agent conversations and state management, letting developers focus on the logic of the task.

*   **Improved Performance and Task Decomposition:** By breaking down a complex problem and assigning parts to specialized agents, AutoGen can often achieve better and more reliable results than a single prompt to one large model.

In short, AutoGen is a strong choice if your project can benefit from multiple AI agents collaborating, requires a mix of automation and human oversight, and needs the flexibility to use different models and tools.

TERMINATE

## Cons of AutoGen:
Based on my research, here are the primary disadvantages or "cons" of choosing AutoGen for a project:

*   **Steep Learning Curve for Complex Scenarios:** While getting a simple two-agent conversation running is straightforward, mastering the framework for complex, customized multi-agent workflows can be difficult. The unique abstractions (like group chats, nested chats, and reply functions) require a significant investment to understand and use effectively.

*   **Immature and Evolving Framework:** AutoGen is still under heavy development. This can lead to breaking changes in the API, a lack of stability, and features that may not be fully fleshed out or could be buggy. It may not be as feature-rich as more mature alternatives in certain areas.

*   **Documentation and Debugging Challenges:** Users frequently report that the documentation can be hard to navigate, lacks sufficient real-world examples, and may not keep pace with the rapid development. Furthermore, debugging the flow of conversation between multiple agents can be complex and time-consuming.

*   **Architectural Constraints:** The framework imposes its own specific architectural patterns. Some users have found these constraints to be limiting for their specific use cases and have pointed out that the API can feel inefficient for certain tasks.TERMINATE



## Decision:

Based on the provided research, here is the decision and rationale:

**Decision:** We will conditionally approve the use of AutoGen for the project.

**Rationale:**

The decision to use AutoGen is justified if the project's core requirement is solving a complex problem that benefits from the collaboration of multiple, specialized AI agents. The primary "Pros" highlight that AutoGen is purpose-built for this exact scenario, offering superior task decomposition and human-in-the-loop capabilities that a single-agent approach would lack.

However, this approval is conditional. The project team must be prepared to accept and mitigate the significant risks outlined in the "Cons." Specifically, the project plan must account for a steep learning curve, potential instability due to the framework's immaturity, and the extra time required for debugging complex agent interactions. If the project's requirements are simple and do not necessitate a multi-agent approach, the drawbacks of AutoGen would outweigh its benefits, and it should not be used.

In short, we will use AutoGen for its unique strengths in multi-agent orchestration, but only for a problem complex enough to justify the significant development and stability risks involved.

TERMINATE

In [25]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [26]:
await host.stop()